In [1]:
import os
import psycopg2
import numpy as np
import pandas as pd
import json, ast, time

from importlib import reload

from dotenv import load_dotenv

from pgvector.psycopg2 import register_vector

from utils import api, db, prompt, prompting

from sklearn.metrics.pairwise import cosine_similarity

load_dotenv()

True

In [2]:
reload(prompt)
reload(api)
reload(db)
reload(prompting)

<module 'utils.prompting' from '/Volumes/Personal/Work/Feedloop/fact_ext/from_github/AI-Research/utils/prompting.py'>

In [3]:
total_eval_prompt_token = 0
total_eval_completion_token = 0

# Trial 1

In [17]:
dict_answer = {}

In [3]:
total_eval_prompt_token = 0
total_eval_completion_token = 0

In [4]:
json_path = "./gpt_answer/SentWindow.json"
csv_path = "./eval_result/SentWindow/Health_RAGAS_DIV_CAL1.csv"
doc_name = "FL Healthcare Use Case - Medical Record"
user_language = "Indonesian"

In [22]:
with open(json_path, 'r') as json_file:
        json_answer_data = json.load(json_file)
    
cur_doc_answer = json_answer_data[doc_name]
question_list, answer_list, context_list, truth_list = [], [], [], []
for i in cur_doc_answer:
    question_list.append(i['question'])
    answer_list.append(i['answer'])
    knowledge = ""
    for k in i['contexts']:
        knowledge+=f'{k}\n'
    context_list.append(knowledge)
    
    ground_truth = ""
    for j in i['ground_truths']:
        ground_truth+=f'{j}\n'
    truth_list.append(ground_truth)

json_eval_result = []

In [47]:
# eval_prompt = prompt.get_eval_ragas_cr_detail1(question_list[2], truth_list[2], answer_list[2], user_language)
eval_prompt = prompt.get_eval_ragas_cr_trial1(question_list[2], answer_list[2])
# print(eval_prompt)

In [33]:
eval_completion = api.get_completions_gpt4_turbo([
    {
        "role": "user",
        "content": eval_prompt,
    }
], temp=0)
total_eval_prompt_token+=eval_completion.usage.prompt_tokens
total_eval_completion_token+=eval_completion.usage.completion_tokens

In [34]:
response = eval_completion.choices[0].message.content
try:
    response = ast.literal_eval(response)
except:
    response = ast.literal_eval(response.replace("null", "None"))

In [36]:
print(response["relevancy"])

1


In [96]:
dict_answer["FL_SW_18"] = response

In [121]:
with open('./trial_eval/trialx.json', 'w') as json_file:
    json.dump(response, json_file, indent=2)

In [53]:
print(f"eval prompt token: {total_eval_prompt_token}\neval completion token: {total_eval_completion_token}")
eval_prompt_token_price = total_eval_prompt_token * 0.01 * 0.001
eval_completion_token_price = total_eval_completion_token * 0.03 * 0.001
print(f"eval prompt token price: {round(eval_prompt_token_price,2)}\neval completion token price: {round(eval_completion_token_price,2)}")

eval prompt token: 30865
eval completion token: 11042
eval prompt token price: 0.31
eval completion token price: 0.33


# Function

In [124]:
def trial1(json_path, doc_name):
    global total_eval_prompt_token, total_eval_completion_token
    with open(json_path, 'r') as json_file:
        json_answer_data = json.load(json_file)
    
    cur_doc_answer = json_answer_data[doc_name]
    dict_answer = {}
    question_list, answer_list, context_list, truth_list = [], [], [], []
    for i in cur_doc_answer:
        question_list.append(i['question'])
        answer_list.append(i['answer'])
        knowledge = ""
        for k in i['contexts']:
            knowledge+=f'{k}\n'
        context_list.append(knowledge)
        
        ground_truth = ""
        for j in i['ground_truths']:
            ground_truth+=f'{j}\n'
        truth_list.append(ground_truth)
    
    for i in range(len(question_list)):
        eval_prompt = prompt.get_eval_ragas_pr_detail3(question_list[i], answer_list[i], context_list[i])
        eval_completion = api.get_completions_gpt4_turbo([
            {
            "role": "user",
            "content": eval_prompt,
            }
        ], temp=0)
        total_eval_prompt_token+=eval_completion.usage.prompt_tokens
        total_eval_completion_token+=eval_completion.usage.completion_tokens

        response = eval_completion.choices[0].message.content
        try:
            response = ast.literal_eval(response)
        except:
            response = ast.literal_eval(response.replace("null", "None"))
        
        dict_answer["Case"+str(i)] = response
    
    with open('./trial_eval/trial1_no_example1.json', 'w') as json_file:
        json.dump(dict_answer, json_file, indent=2)

In [125]:
trial1(json_path, doc_name)

In [5]:
with open('./trial_eval/trial1_no_example1.json', 'r') as json_file:
    dict_x = json.load(json_file)

In [6]:
df_data = []
for case, values in dict_x.items():
    total_attributed = sum(item['Attributed'] for item in values['classification'])
    classification_length = len(values['classification'])
    recall = total_attributed / classification_length
    precision = values['verification']['verdict']
    df_data.append({'Case': case, 'Recall': recall, 'Precision': precision})

df = pd.DataFrame(df_data)

In [7]:
print(df)

      Case  Recall  Precision
0    Case0     1.0          1
1    Case1     1.0          1
2    Case2     1.0          1
3    Case3     1.0          1
4    Case4     0.0          0
5    Case5     1.0          1
6    Case6     1.0          1
7    Case7     1.0          1
8    Case8     0.0          0
9    Case9     1.0          1
10  Case10     1.0          1
11  Case11     1.0          1
12  Case12     1.0          0
13  Case13     1.0          1
14  Case14     0.0          0
15  Case15     1.0          1
16  Case16     1.0          1
17  Case17     1.0          1
18  Case18     1.0          1
19  Case19     1.0          1


## Relevancy and correctness

In [54]:
def trial2(json_path, doc_name, user_language):
    global total_eval_prompt_token, total_eval_completion_token
    with open(json_path, 'r') as json_file:
        json_answer_data = json.load(json_file)
    
    cur_doc_answer = json_answer_data[doc_name]
    dict_answer = {}
    question_list, answer_list, context_list, truth_list = [], [], [], []
    for i in cur_doc_answer:
        question_list.append(i['question'])
        answer_list.append(i['answer'])
        knowledge = ""
        for k in i['contexts']:
            knowledge+=f'{k}\n'
        context_list.append(knowledge)
        
        ground_truth = ""
        for j in i['ground_truths']:
            ground_truth+=f'{j}\n'
        truth_list.append(ground_truth)

    for i in range(len(question_list)):
        eval_prompt = prompt.get_eval_ragas_cr_detail1(question_list[i], truth_list[i], answer_list[i])
        eval_completion = api.get_completions_gpt4_turbo([
            {
            "role": "user",
            "content": eval_prompt,
            }
        ], temp=0)
        total_eval_prompt_token+=eval_completion.usage.prompt_tokens
        total_eval_completion_token+=eval_completion.usage.completion_tokens

        response = eval_completion.choices[0].message.content
        try:
            response = ast.literal_eval(response)
        except:
            response = ast.literal_eval(response.replace("null", "None"))
        
        # eval_prompt_r = prompt.get_eval_ragas_cr_trial1(question_list[i], answer_list[i])
        # eval_completion_r = api.get_completions_gpt4_turbo([
        #     {
        #     "role": "user",
        #     "content": eval_prompt_r,
        #     }
        # ], temp=0)
        # total_eval_prompt_token+=eval_completion_r.usage.prompt_tokens
        # total_eval_completion_token+=eval_completion_r.usage.completion_tokens

        # response_r = eval_completion_r.choices[0].message.content
        # try:
        #     response_r = ast.literal_eval(response_r)
        # except:
        #     response_r = ast.literal_eval(response_r.replace("null", "None"))
        
        response["question"] = question_list[i]
        # response["relevance"] = response_r["relevancy"]
        dict_answer["Case"+str(i)] = response
    
    with open('./trial_eval/trial_rc_no_example1d.json', 'w') as json_file:
        json.dump(dict_answer, json_file, indent=2)

In [50]:
print(int(not bool(1)))

0


In [55]:
trial2(json_path, doc_name, user_language)

In [56]:
with open('./trial_eval/trial_rc_no_example1d.json', 'r') as json_file:
    dict_cr = json.load(json_file)

In [59]:
list_rel = [int(not bool(data["relevancy"])) for case, data in dict_cr.items()]

In [184]:
correctness_values = {case: calculate_correctness(data['tp'], data['fp'], data['fn']) for case, data in dict_cr.items()}
df['Correctness'] = df['Case'].map(correctness_values)
sim_values = {case: calculate_similarity(data['question'], data['gen_question'], data['committal']) for case, data in dict_cr.items()}
df['Relevancy'] = df['Case'].map(sim_values)

In [188]:
print(df)

      Case  Recall  Precision  Correctness  Relevancy
0    Case0     1.0          1     1.000000      0.986
1    Case1     1.0          1     1.000000      0.954
2    Case2     1.0          1     0.000000      0.000
3    Case3     1.0          1     0.857143      0.000
4    Case4     0.0          0     0.000000      0.000
5    Case5     1.0          1     0.666667      0.969
6    Case6     1.0          1     0.000000      0.000
7    Case7     1.0          1     0.000000      0.000
8    Case8     0.0          0     0.000000      0.000
9    Case9     1.0          1     1.000000      0.986
10  Case10     1.0          1     0.750000      0.980
11  Case11     1.0          1     0.000000      0.000
12  Case12     1.0          0     0.000000      0.000
13  Case13     1.0          1     0.000000      0.000
14  Case14     0.0          0     0.000000      0.000
15  Case15     1.0          1     1.000000      0.996
16  Case16     1.0          1     0.000000      0.000
17  Case17     1.0          

In [140]:
cr_resp = trial2(json_path, doc_name, user_language)

In [143]:
cr_resp1 = trial2(json_path, doc_name, user_language)

In [141]:
print(cr_resp.choices[0].message.content)

{
    "gen_question": "Kapan jadwal pertemuan Ahmad Surya dengan Dr. Indah Pratiwi selanjutnya untuk evaluasi pengobatan?",
    "committal": 1,
    "tp": [
        "Pertemuan berikutnya antara Ahmad Surya dengan Dr. Indah Pratiwi untuk evaluasi pengobatan",
        "Pertemuan dijadwalkan pada tanggal 5 Desember 2023"
    ],
    "fp": [],
    "fn": []
}


In [144]:
print(cr_resp1.choices[0].message.content)

{
    "gen_question": "Kapan dijadwalkannya pertemuan berikutnya antara John Doe dengan Dr. Sarah Smith?",
    "committal": "0",
    "tp": [],
    "fp": ["Maaf, saya tidak menemukan informasi tentang pertemuan berikutnya antara John Doe dengan Dr. Sarah Smith dalam dokumen yang disediakan."],
    "fn": ["Tanggal pertemuan berikutnya antara John Doe dengan Dr. Sarah Smith adalah 15 Agustus 2023."]
}


# Implementation

In [4]:
def generate_list(cur_doc_answer):
    question_list, answer_list, context_list, truth_list = [], [], [], []
    for i in cur_doc_answer:
        question_list.append(i['question'])
        answer_list.append(i['answer'])
        knowledge = ""
        for k in i['contexts']:
            knowledge+=f'{k}\n'
        context_list.append(knowledge)
        
        ground_truth = ""
        for j in i['ground_truths']:
            ground_truth+=f'{j}\n'
        truth_list.append(ground_truth)
    return question_list, answer_list, context_list, truth_list

In [5]:
def increase_token_usage(completion):
    global total_eval_prompt_token, total_eval_completion_token
    total_eval_prompt_token+=completion.usage.prompt_tokens
    total_eval_completion_token+=completion.usage.completion_tokens

In [6]:
def run_model(eval_prompt):
    return api.get_completions_gpt4_turbo([
        {
        "role": "user",
        "content": eval_prompt,
        }
    ], temp=0)

In [7]:
def convert_to_json(response):
    try:
        res_json = ast.literal_eval(response)
    except:
        res_json = ast.literal_eval(response.replace("null", "None"))
    return res_json

In [8]:
def run_eval_pr(question, answer, context):
    eval_prompt = prompt.get_eval_ragas_pr_detail3(question, answer, context)
    eval_completion = run_model(eval_prompt)
    
    increase_token_usage(eval_completion)

    return convert_to_json(eval_completion.choices[0].message.content)

In [9]:
def run_eval_cr(question, truth, answer):
    eval_prompt = prompt.get_eval_ragas_cr_detail1(question, truth, answer)
    eval_completion = run_model(eval_prompt)
    
    increase_token_usage(eval_completion)

    return convert_to_json(eval_completion.choices[0].message.content)

In [10]:
def calculate_correctness(tp, fp, fn):
    tp_count = len(tp)
    fp_count = len(fp)
    fn_count = len(fn)
    return round(tp_count / (tp_count + 0.5 * (fp_count + fn_count)), 2) if tp_count > 0 else 0

In [11]:
def calculate_similarity(question, gen_question, noncommittal):
    if isinstance(noncommittal, str):
        noncommittal = int(noncommittal)
    if noncommittal == 1:
        return 0
    embedding1_np = np.array(api.get_embeddings_ada(question))
    embedding2_np = np.array(api.get_embeddings_ada(gen_question))
    return round(cosine_similarity([embedding1_np], [embedding2_np])[0][0],3)

In [12]:
def generate_df(response):
    df_data = []
    for _, values in response.items():
        total_attributed = sum(item['Attributed'] for item in values['classification'])
        classification_length = len(values['classification'])
        recall = total_attributed / classification_length
        
        precision = values['verification']['verdict']

        correctness = calculate_correctness(values['tp'], values['fp'], values['fn'])
    
        similarity = calculate_similarity(values['question'], values['gen_question'], values['relevancy'])

        df_data.append({'Context Recall': recall, 'Context Precision': precision, 'Answer Correctness': correctness, 'Answer Relevance': similarity})


    return pd.DataFrame(df_data)

In [21]:
def final_eval(json_answer_data, doc_name, csv_path, json_output_path):
    
    # with open(json_path, 'r') as json_file:
    #     json_answer_data = json.load(json_file)
    
    cur_doc_answer = json_answer_data[doc_name]
    dict_res = {}
    question_list, answer_list, context_list, truth_list = generate_list(cur_doc_answer)
    
    for i in range(len(question_list)):
        response_pr = run_eval_pr(question_list[i], truth_list[i], context_list[i])
        response_cr = run_eval_cr(question_list[i], truth_list[i], answer_list[i])
        combined_res = {**response_pr, **response_cr}
        combined_res["question"] = question_list[i]
        
        dict_res["Case"+str(i)] = combined_res
        time.sleep(10)
    
    with open(json_output_path, 'w') as json_file:
        json.dump(dict_res, json_file, indent=2)
    # df = generate_df(dict_res)
    # df.to_csv(csv_path)

In [23]:
json_path = './gpt_answer/GPT35_ParentChild.json'
with open(json_path, 'r') as json_file:
    dict_gpt_answer = json.load(json_file)

for key in dict_gpt_answer:
    if key == "Perbup Sleman Nomor 11.3 Tahun 2019 ttg Pengelolaan keuangan desa":
        continue
    json_output_path = f"./new_eval_res/GPT35_ParentChild/json/{key}.json"
    csv_path = f"./new_eval_res/GPT35_ParentChild/csv/{key}.csv"
    doc_name = key
    final_eval(dict_gpt_answer, doc_name, csv_path, json_output_path)


In [24]:
def generate_csv(method, dict_gpt_answer):
    for key in dict_gpt_answer:
        if key == "Perbup Sleman Nomor 11.3 Tahun 2019 ttg Pengelolaan keuangan desa":
            continue
        with open(f'./new_eval_res/{method}/json/{key}.json', 'r') as json_file:
            dict_temp = json.load(json_file)
        df_temp = generate_df(dict_temp)
        df_temp.to_csv(f'./new_eval_res/{method}/csv/{key}.csv')

In [25]:
generate_csv("GPT35_ParentChild", dict_gpt_answer)

In [15]:
print(f"eval prompt token: {total_eval_prompt_token}\neval completion token: {total_eval_completion_token}")
eval_prompt_token_price = total_eval_prompt_token * 0.01 * 0.001
eval_completion_token_price = total_eval_completion_token * 0.03 * 0.001
print(f"eval prompt token price: {round(eval_prompt_token_price,2)}\neval completion token price: {round(eval_completion_token_price,2)}")

eval prompt token: 20258
eval completion token: 6234
eval prompt token price: 0.2
eval completion token price: 0.19
